# More LSOA and Parish Sketches

Exploring other routes to get LSOAs, Parish data, etc.

One convenient, albeiut unoffical and *ad hoc*, source of UK shapefiles is: https://github.com/drkane/geo-lookups/ 

For example, we can get LSOAs:

- index data: https://raw.githubusercontent.com/drkane/geo-lookups/refs/heads/master/lsoa_la.csv
- LSOA boundaries: https://raw.githubusercontent.com/drkane/geo-lookups/refs/heads/master/boundaries/lsoa/E06000001.geojson

In [ ]:
import polars as pl


def get_dr_kane_lsoa_la_lookups():
    geo_index_url = "https://raw.githubusercontent.com/drkane/geo-lookups/refs/heads/master/lsoa_la.csv"

    lsoa_df = pl.scan_csv(geo_index_url)
    return lsoa_df


lsoa_df = get_dr_kane_lsoa_la_lookups()
lsoa_df.head()

In [ ]:
# Get LAD code from LA Name


def getLADcodeFromName(df):
    return (
        df.select(["LADNM_ACTIVE", "LADCD_ACTIVE"])
        .filter(pl.col("LADNM_ACTIVE") == "Isle of Wight")
        .collect()
        .unique()
    )

In [ ]:
getLADcodeFromName(lsoa_df)

LADNM_ACTIVE,LADCD_ACTIVE
str,str
"""Isle of Wight""","""E06000046"""


In [ ]:
def get_dr_kane_wards_lookups():
    geo_index_url = "https://raw.githubusercontent.com/drkane/geo-lookups/refs/heads/master/ward_all_codes.csv"

    wards_df = pl.scan_csv(geo_index_url)
    return wards_df


wards_df = get_dr_kane_wards_lookups()
wards_df.head()

In [81]:
getLADcodeFromName(wards_df)

LADNM_ACTIVE,LADCD_ACTIVE
str,str
"""Isle of Wight""","""E06000046"""


If we know the LSOA or ward code, we can get the boundary file:

In [ ]:
import requests
from shapely.geometry import shape
import geopandas as gpd


def get_drkane_geoboundary(id, typ="LSOA", name="LSOA21NM", astype="gpd"):
    """Simple function to pull LSOA data from drkane's geo repo."""
    base_url = "https://raw.githubusercontent.com/drkane/geo-lookups/refs/heads/master/boundaries/"
    url = ""
    if typ == "LSOA":
        url = f"{base_url}lsoa/{id}.geojson"
    elif typ == "ward":
        url = f"{base_url}ward/{id}.geojson"
    if url:
        if astype == "gpd":
            return gpd.read_file(url)
        geojson = requests.get(url).json()
        if astype == "polygon":
            geometries = {
                feature["properties"][name]: shape(feature["geometry"])
                for feature in geojson["features"]
            }
            return geometries
        return geojson

In [82]:
# Example of getting the LSOA boundaries within an LA
LA_code = getLADcodeFromName(lsoa_df)[0, "LADCD_ACTIVE"]
gj_data = get_drkane_geoboundary(LA_code, astype="gpd")
gj_data.head(3)

https://raw.githubusercontent.com/drkane/geo-lookups/refs/heads/master/boundaries/lsoa/E06000046.geojson


,FID,LSOA21CD,LSOA21NM,GlobalID,LADCD_ACTIVE,UTLACD,CAUTHCD,RGNCD,geometry
0,16412,E01017282,Isle of Wight 006A,2c7fa55d-6a50-4f9a-83a2-4660101080bf,E06000046,E06000046,None,E12000008,"POLYGON ((-1.17166 50.72115, -1.17089 50.72046..."
1,16413,E01017283,Isle of Wight 006B,513668ed-b2fb-4e5c-b850-0249d34382b3,E06000046,E06000046,None,E12000008,"POLYGON ((-1.16807 50.72136, -1.16972 50.72077..."
2,16414,E01017284,Isle of Wight 010A,888accb6-dd1e-4e96-8095-632580d4be2c,E06000046,E06000046,None,E12000008,"POLYGON ((-1.07777 50.69221, -1.08034 50.69064..."


In [84]:
# Example of getting the Ward boundaries within an LAD
gj_data2 = get_drkane_geoboundary(LA_code, typ="ward", astype="gpd")
gj_data2.head()

https://raw.githubusercontent.com/drkane/geo-lookups/refs/heads/master/boundaries/ward/E06000046.geojson


,WD22CD,WD22NM,WD22NMW,LAD22CD,LAD22NM,BNG_E,BNG_N,LONG,LAT,FID,GlobalID,LADCD_ACTIVE,UTLACD,CAUTHCD,RGNCD,geometry
0,E05013359,Bembridge,,E06000046,Isle of Wight,463510,87135,-1.10243,50.68040,5873,491b4aff-1c14-49fb-a3bd-b382f7085d6a,E06000046,E06000046,None,E12000008,"POLYGON ((-1.10896 50.66555, -1.11694 50.66866..."
1,E05013360,Binstead & Fishbourne,,E06000046,Isle of Wight,456304,92223,-1.20364,50.72689,5874,ff2026dd-877e-48de-97e2-26fe8f0b395d,E06000046,E06000046,None,E12000008,"POLYGON ((-1.17441 50.73368, -1.17438 50.73257..."
2,E05013361,Brading & St Helens,,E06000046,Isle of Wight,460289,87763,-1.14792,50.68639,5875,84707b29-b066-4f73-8dce-d251701a35d9,E06000046,E06000046,None,E12000008,"POLYGON ((-1.14694 50.70689, -1.14114 50.7053,..."
3,E05013362,"Brighstone, Calbourne & Shalfleet",,E06000046,Isle of Wight,442609,85798,-1.39840,50.67028,5876,8b8cc5a9-aad5-474a-9392-5ece115ec37a,E06000046,E06000046,None,E12000008,"MULTIPOLYGON (((-1.42653 50.70598, -1.42656 50..."
4,E05013363,Carisbrooke & Gunville,,E06000046,Isle of Wight,447075,86548,-1.33511,50.67668,5877,7b5bfd02-61ab-44a3-8bd4-41df048a036f,E06000046,E06000046,None,E12000008,"POLYGON ((-1.32014 50.70135, -1.32141 50.70066..."
